In [3]:
import tkinter as tk
import openpyxl
from tkinter import messagebox
from tkinter import ttk
import pickle
import pandas as pd
import smtplib
from openpyxl import load_workbook
from tkinter import messagebox

# Modelin formülü
def make_prediction(marka, model, arac_paketi, motor_gucu, motor_hacmi):
    return 873592.76 + (1304064.97 * motor_gucu) + (543824.59 * motor_hacmi) + (961990.11 * marka) + (-92.35 * arac_paketi) + (-9.29 * model)

# Değerlerin sözlükleri
markalar = {'Hyundai': 0, 'Porsche': 1}
modeller = {'Bayon': -9999, 'Kona': 1, 'Panamera': 2, 'Taycan': 3}
arac_paketleri = {'4 Cross Turismo': -9999, '4S': -9999, '4S Cross Turismo': -9999, '4S Performance': 3, '4S Performance Plus': -9999, '4S Sport Turismo': 5, 'Elite': 6, 'Elite Smart': 7, 'GTS': 8, 'GTS Sport Turismo': 9, 'Hibrit Elite': 10, 'Hibrit Smart': 11, 'Hibrit Style': 12, 'Jump': 13, 'Panamera': 14, 'Panamera 4': 15, 'Panamera 4 - 10 Years Edition': 16, 'Panamera 4 E-Hybrid': 17, 'Panamera 4 Platinum Edition': 18, 'Panamera 4 Sport Turismo': 19, 'Panamera 4S': 20, 'Panamera 4S Diesel': 21, 'Panamera 4S E-Hybrid': 22, 'Panamera 4S Sport Turismo': 23, 'Panamera Diesel': 24, 'Panamera GTS': 25, 'Panamera S': 26, 'Panamera S Hybrid': 27, 'Panamera Turbo': -9999, 'Panamera Turbo S': 29, 'Panamera Turbo S E-Hybrid': 30, 'Progressive': 31, 'Smart': 32, 'Style': 33, 'Style Design': 34, 'Taycan': -9999, 'Turbo': 36, 'Turbo S': -9999, 'Turbo S Cross Turismo': 38}
motor_gucu_secenekleri = {'200 hp\'e kadar': 0, '200-500 hp': 1, '500 hp\'ten fazla': 2}
motor_hacimleri = {'1600 cm³’e kadar': 0, '2000 cm³’ten fazla': 1, 'Elektrikli Motor': 2}

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Tahmin Uygulaması")

        # Marka Combobox
        self.marka_label = tk.Label(self, text="Marka:")
        self.marka_label.pack()
        self.marka_combobox = ttk.Combobox(self, values=list(markalar.keys()), state="readonly")
        self.marka_combobox.pack()
        self.marka_combobox.bind("<<ComboboxSelected>>", self.on_marka_select)

        # Model Combobox
        self.model_label = tk.Label(self, text="Model:")
        self.model_label.pack()
        self.model_combobox = ttk.Combobox(self, state="disabled")
        self.model_combobox.pack()
        self.model_combobox.bind("<<ComboboxSelected>>", self.on_model_select)

        # Araç Paketi Combobox
        self.arac_paketi_label = tk.Label(self, text="Araç Paketi:")
        self.arac_paketi_label.pack()
        self.arac_paketi_combobox = ttk.Combobox(self, state="disabled")
        self.arac_paketi_combobox.pack()
        self.arac_paketi_combobox.bind("<<ComboboxSelected>>", self.on_arac_paketi_select)

        # Motor Gücü Combobox
        self.motor_gucu_label = tk.Label(self, text="Motor Gücü:")
        self.motor_gucu_label.pack()
        self.motor_gucu_combobox = ttk.Combobox(self, values=list(motor_gucu_secenekleri.keys()), state="disabled")
        self.motor_gucu_combobox.pack()

        # Motor Hacmi Combobox
        self.motor_hacmi_label = tk.Label(self, text="Motor Hacmi:")
        self.motor_hacmi_label.pack()
        self.motor_hacmi_combobox = ttk.Combobox(self, values=list(motor_hacimleri.keys()), state="disabled")
        self.motor_hacmi_combobox.pack()

        # Tahmin Butonu
        self.tahmin_button = tk.Button(self, text="Tahmin Yap", command=self.predict)
        self.tahmin_button.pack()
        
        # geri bildirim Butonu
        self.open_feedback_form = tk.Button(self, text="geribildirim yap", command=self.form_ac)
        self.open_feedback_form.pack()
        
        
    def form_ac(self):
        feedback_form = tk.Toplevel(self)

        # Etiketler
        ad_label = tk.Label(feedback_form, text="Adınız:")
        ad_label.grid(row=0, column=0, sticky="e")

        email_label = tk.Label(feedback_form, text="E-posta Adresiniz:")
        email_label.grid(row=1, column=0, sticky="e")

        mesaj_label = tk.Label(feedback_form, text="Mesajınız:")
        mesaj_label.grid(row=2, column=0, sticky="e")

        # Giriş alanları
        self.ad_entry = tk.Entry(feedback_form)
        self.ad_entry.grid(row=0, column=1)

        self.email_entry = tk.Entry(feedback_form)
        self.email_entry.grid(row=1, column=1)

        self.mesaj_text = tk.Text(feedback_form, height=5)
        self.mesaj_text.grid(row=2, column=1)

        # Gönder düğmesi
        submit_button = tk.Button(feedback_form, text="Gönder", command=self.submit_feedback)
        submit_button.grid(row=3, column=0, columnspan=2)

        # Geri bildirimleri göster düğmesi
        view_feedback_button = tk.Button(feedback_form, text="Geri Bildirimleri Göster", command=self.view_feedback)
        view_feedback_button.grid(row=4, column=0, columnspan=2)

        # Bilgi etiketi
        self.info_label = tk.Label(feedback_form, text="")
        self.info_label.grid(row=5, column=0, columnspan=2)

    def submit_feedback(self):
        ad = self.ad_entry.get()
        email = self.email_entry.get()
        mesaj = self.mesaj_text.get("1.0", "end")

        # Excel dosyasını aç ve son satırı bul
        workbook = openpyxl.load_workbook("geribildirim.xlsx")
        sheet = workbook.active
        last_row = sheet.max_row + 1

        # Yeni geri bildirimi Excel dosyasına ekle
        sheet.cell(row=last_row, column=1, value=ad)
        sheet.cell(row=last_row, column=2, value=email)
        sheet.cell(row=last_row, column=3, value=mesaj)

        # Excel dosyasını kaydet
        workbook.save("geribildirim.xlsx")

        # Geri bildirimin alındığını gösteren bir mesaj penceresi oluştur
        self.info_label.config(text="Geri bildiriminiz alındı. Teşekkür ederiz!")

    def view_feedback(self):
        if os.path.exists("geribildirim.xlsx"):
            os.system("start geribildirim.xlsx")
        else:
            self.info_label.config(text="Kaydedilmiş geri bildirim bulunamadı!")

        


    def on_marka_select(self, event):
        marka = self.marka_combobox.get()
        if marka == "Hyundai":
            self.model_combobox["values"] = ["Bayon", "Kona"]
        elif marka == "Porsche":
            self.model_combobox["values"] = ["Panamera", "Taycan"]
        self.model_combobox.config(state="readonly")
        self.model_combobox.set("")
        self.arac_paketi_combobox.config(state="disabled")
        self.motor_gucu_combobox.config(state="disabled")
        self.motor_hacmi_combobox.config(state="disabled")

    def on_model_select(self, event):
        selected_model = self.model_combobox.get()

        if selected_model == 'Bayon':
            self.arac_paketi_combobox['values'] = ['Elite', 'Jump', 'Style', 'Style Design']
            self.motor_gucu_combobox['values'] = ['200 hp\'e kadar']
            self.motor_hacmi_combobox['values'] = ['1600 cm³’e kadar']
        elif selected_model == 'Kona':
            self.arac_paketi_combobox['values'] = ['Elite', 'Elite Smart', 'Style', 'Smart', 'Hibrit Elite', 'Hibrit Smart', 'Hibrit Style', 'Progressive']
        elif selected_model == 'Taycan':
            self.arac_paketi_combobox['values'] = ['4S', '4S Cross Turismo', '4S Performance', '4S Performance Plus', 'GTS', 'GTS Sport Turismo', 'Turbo', 'Turbo S Cross Turismo', '4 Cross Turismo', '4S Sport Turismo', 'Taycan']
        elif selected_model == 'Panamera':
            self.arac_paketi_combobox['values'] = ['Panamera', 'Panamera 4', 'Panamera 4 - 10 Years Edition', 'Panamera 4 E-Hybrid', 'Panamera 4 Platinum Edition', 'Panamera 4 Sport Turismo', 'Panamera 4S', 'Panamera 4S Diesel', 'Panamera 4S E-Hybrid', 'Panamera 4S Sport Turismo', 'Panamera Diesel', 'Panamera GTS', 'Panamera S', 'Panamera S Hybrid', 'Panamera Turbo', 'Panamera Turbo S', 'Panamera Turbo S E-Hybrid']
        else:
            self.arac_paketi_combobox['values'] = []
            self.motor_gucu_combobox['values'] = []
            self.motor_hacmi_combobox['values'] = []

        self.arac_paketi_combobox.config(state="readonly")
        self.arac_paketi_combobox.set("")
        self.motor_gucu_combobox.config(state="disabled")
        self.motor_hacmi_combobox.config(state="disabled")

    def on_arac_paketi_select(self, event):
        selected_model = self.model_combobox.get()
        selected_arac_paketi = self.arac_paketi_combobox.get()

        if selected_model == 'Bayon':
            if selected_arac_paketi in ['Elite', 'Jump', 'Style', 'Style Design']:
                self.motor_gucu_combobox['values'] = ['200 hp\'e kadar']
                self.motor_hacmi_combobox['values'] = ['1600 cm³’e kadar']
            else:
                self.motor_gucu_combobox['values'] = []
                self.motor_hacmi_combobox['values'] = []
        elif selected_model == 'Kona':
            if selected_arac_paketi == 'Elite':
                self.motor_gucu_combobox['values'] = ['200-500 hp', '200 hp\'e kadar']
                self.motor_hacmi_combobox['values'] = ['Elektrikli Motor', '1600 cm³’e kadar']
            elif selected_arac_paketi =='Elite Smart':
                self.motor_gucu_combobox['values'] = ['200-500 hp','200 hp\'e kadar']
                self.motor_hacmi_combobox['values'] = ['1600 cm³’e kadar']
            elif selected_arac_paketi in ['Style', 'Smart', 'Hibrit Elite', 'Hibrit Smart', 'Hibrit Style']:
                self.motor_gucu_combobox['values'] = ['200 hp\'e kadar']
                self.motor_hacmi_combobox['values'] = ['1600 cm³’e kadar']
            elif selected_arac_paketi == 'Progressive':
                self.motor_gucu_combobox['values'] = ['200-500 hp']
                self.motor_hacmi_combobox['values'] = ['Elektrikli Motor']
            else:
                self.motor_gucu_combobox['values'] = []
                self.motor_hacmi_combobox['values'] = []
        elif selected_model == 'Taycan':
            if selected_arac_paketi in ['4 Cross Turismo', '4S Sport Turismo', 'Taycan']:
                self.motor_gucu_combobox['values'] = ['200-500 hp']
            elif selected_arac_paketi in ['4S', '4S Cross Turismo', '4S Performance', '4S Performance Plus', 'GTS', 'GTS Sport Turismo', 'Turbo S Cross Turismo']:
                self.motor_gucu_combobox['values'] = ['500 hp\'ten fazla']
            elif selected_arac_paketi in ['Turbo', 'Turbo S']:
                self.motor_gucu_combobox['values'] = ['200-500 hp', '500 hp\'ten fazla']
            else:
                self.motor_gucu_combobox['values'] = []

            self.motor_hacmi_combobox['values'] = ['Elektrikli Motor']
        elif selected_model == 'Panamera':
            if selected_arac_paketi in ['Panamera', 'Panamera 4', 'Panamera 4 - 10 Years Edition', 'Panamera 4 E-Hybrid', 'Panamera 4 Platinum Edition', 'Panamera 4 Sport Turismo', 'Panamera 4S', 'Panamera 4S Diesel', 'Panamera 4S Sport Turismo', 'Panamera Diesel', 'Panamera GTS', 'Panamera S', 'Panamera S Hybrid', 'Panamera Turbo']:
                self.motor_gucu_combobox['values'] = ['200-500 hp']
            elif selected_arac_paketi in ['Panamera 4S E-Hybrid', 'Panamera Turbo S E-Hybrid', 'Panamera Turbo S']:
                self.motor_gucu_combobox['values'] = ['500 hp\'ten fazla']
            elif selected_arac_paketi in ['Panamera GTS', 'Panamera Turbo']:
                self.motor_gucu_combobox['values'] = ['500 hp\'ten fazla', '200-500 hp']
            else:
                self.motor_gucu_combobox['values'] = []

            self.motor_hacmi_combobox['values'] = ['2000 cm³’ten fazla']
        else:
            self.motor_gucu_combobox['values'] = []
            self.motor_hacmi_combobox['values'] = []

        self.motor_gucu_combobox.config(state="readonly")
        self.motor_gucu_combobox.set("")
        self.motor_hacmi_combobox.config(state="readonly")
        self.motor_hacmi_combobox.set("")



    def predict(self):
        marka = markalar.get(self.marka_combobox.get())
        model = modeller.get(self.model_combobox.get())
        arac_paketi = arac_paketleri.get(self.arac_paketi_combobox.get())
        motor_gucu = motor_gucu_secenekleri.get(self.motor_gucu_combobox.get())
        motor_hacmi = motor_hacimleri.get(self.motor_hacmi_combobox.get())

        # Giriş değerlerinin doğruluğunu kontrol et
        if marka is None or model is None or arac_paketi is None or motor_gucu is None or motor_hacmi is None:
            messagebox.showerror("Hata", "Geçersiz giriş değerleri!")
            return

        # Tahmini hesapla
        tahmin = make_prediction(marka, model, arac_paketi, motor_gucu, motor_hacmi)

        # Tahmin sonucunu kontrol et ve -1 ile çarp
        if tahmin < 0:
            tahmin *= -1

        messagebox.showinfo("Tahmin Sonucu", f"Araç Tahmini: {tahmin:.2f}")
    
        
        
        
app = Application()
app.mainloop()

